In [0]:
from cni_connectors import adls_gen1_connector as adls_conn
var_adls_uri = adls_conn.adls_gen1_connect(spark, dbutils, scope="adls_gen2", dynamic_overwrite="dynamic")

In [0]:
import json
import re
import pyspark.sql.functions as f
from pyspark.sql import DataFrame
from pyspark.sql.window import Window
import pyspark.sql.functions as f
from trs_control_field import trs_control_field as tcf

In [0]:
table = {"copy_sqldw":"false",
 "socios_bruto":"rfb_cnpj/cadastro_socio_f",
 "nomes_bruto":"oni/bases_do_projeto/mulheres_industria",
 "estbl_bruto":"rfb_cnpj/cadastro_estbl_f",
 "df_uf":"oni/ibge/uf",
 "path_destination":"oni/mulheres_no_mercado_de_trabalho/rfb",
 "destination":"/mulheres_no_mercado_de_trabalho/rfb",
 "databricks":{"notebook":"/biz/oni/mulheres_no_mercado_de_trabalho/trs_biz_receita"}}

adf = {"adf_factory_name":"cnibigdatafactory","adf_pipeline_name":"trs_biz_rfb_cno","adf_pipeline_run_id":"c158e9dd-98df-4f7b-a64d-d13c76669868","adf_trigger_id":"67c514a7245449b984eb4aadd55bfbff","adf_trigger_name":"Sandbox","adf_trigger_time":"2023-08-22T21:22:42.5769844Z","adf_trigger_type":"Manual"}
 
dls = {"folders":{"landing":"/tmp/dev/lnd","error":"/tmp/dev/err","archive":"/tmp/dev/ach","staging":"/tmp/dev/stg","log":"/tmp/dev/log","raw":"/tmp/dev/raw","trusted":"/tmp/dev/trs","business":"/tmp/dev/biz","prm":"/tmp/dev/prm","historico":"/tmp/dev/hst"},"path_prefix":"tmp","uld":{"folders":{"landing":"/tmp/dev/uld","error":"/tmp/dev/err","staging":"/tmp/dev/stg","log":"/tmp/dev/log","raw":"/tmp/dev/raw","archive":"/tmp/dev/ach"},"systems":{"raw":"usr"},"path_prefix":"/tmp/dev/"},"systems":{"raw":"usr"}}

#cadastro_estbl_f = "{adl_path}{trs}/{file_folder}/".format(adl_path=var_adls_uri, trs=dls["folders"]["trusted"], file_folder=table["cadastro_estbl_f"])
#cadastro_estbl = spark.read.format("parquet").option("header","true").option("encoding", "ISO-8859-1").load(cadastro_estbl_f, mode="FAILFAST")



In [0]:
socios_bruto = "{adl_path}{trusted}/{file_folder}/".format(adl_path=var_adls_uri, trusted=dls["folders"]["trusted"], file_folder=table["socios_bruto"])
nomes_bruto = "{adl_path}{trusted}/{file_folder}/".format(adl_path=var_adls_uri, trusted=dls["folders"]["trusted"], file_folder=table["nomes_bruto"])
estbl_bruto = "{adl_path}{trusted}/{file_folder}/".format(adl_path=var_adls_uri, trusted=dls["folders"]["trusted"], file_folder=table["estbl_bruto"])
df_uf = "{adl_path}{trusted}/{file_folder}/".format(adl_path=var_adls_uri, trusted=dls["folders"]["trusted"], file_folder=table["df_uf"])

In [0]:
# Carrega dados do data lake
socios_bruto = spark.read.parquet(socios_bruto)
nomes_bruto = spark.read.parquet(nomes_bruto)
estbl_bruto = spark.read.parquet(estbl_bruto)
df_uf = spark.read.parquet(df_uf)

In [0]:
estbl_bruto.withColumn('CD_SIT_CADASTRAL', f.col('CD_SIT_CADASTRAL').cast('integer')).select('CD_SIT_CADASTRAL','DS_SIT_CADASTRAL').distinct().show()

In [0]:
nomes = nomes_bruto \
  .fillna(0, ['frequency_female', 'frequency_male']) \
  .withColumn('prop_mulher', f.col('frequency_female')/(f.col('frequency_female') + f.col('frequency_male')))

estbl = estbl_bruto \
  .where(f.col('DS_MATRIZ_FILIAL') == 'Matriz') \
  .select('CD_CNPJ_BASICO', 'SG_UF', 'CD_SIT_CADASTRAL', 'CD_CNAE20_SUBCLASSE_PRINCIPAL') \
  .distinct() \
  .where(~((f.col('CD_CNPJ_BASICO').isin(['42938862'])) & (f.col('CD_CNAE20_SUBCLASSE_PRINCIPAL') != '0115600')))

socios = socios_bruto \
  .withColumn('first_name', f.regexp_extract(f.col('NM_SOCIO_RAZAO_SOCIAL'), '^[A-Z]+', 0)) \
  .join(nomes, on = 'first_name', how = 'left') \
  .join(estbl, on = 'CD_CNPJ_BASICO', how = 'left')

media_mulheres = socios.select(f.mean(socios.prop_mulher)).collect()[0][0]

socios = (socios \
  .fillna(media_mulheres, 'prop_mulher') \
  .withColumn('prop_homem', 1 - f.col('prop_mulher')) \
  .withColumn('CNAE', f.regexp_extract(f.col('CD_CNAE20_SUBCLASSE_PRINCIPAL'), '^.{2}', 0)) \
  .withColumn('ANO', f.year('DT_ENTRADA_SOCIEDADE')) \
  .withColumn('CD_SIT_CADASTRAL',f.col('CD_SIT_CADASTRAL').cast('integer'))
  .withColumn('cd_sit_cadastral', f.when(f.col('CD_SIT_CADASTRAL')==f.lit(2), f.lit(1)).otherwise(f.when(f.col('CD_SIT_CADASTRAL').isin([1,4,3,8]),f.lit(2)).otherwise(f.lit(3)))) \
  .withColumn('ds_sit_cadastral', f.when(f.col('CD_SIT_CADASTRAL')==f.lit(2), f.lit("Ativa")).otherwise(f.when(f.col('CD_SIT_CADASTRAL').isin([1,4,3,8]),f.lit("Inativa")).otherwise(f.lit("Não informado"))))
  .withColumn('ds_faixa_etaria_socio', f.when(f.col('CD_FAIXA_ETARIA_SOCIO').isin([1,2]), f.lit('Até 20 anos')).otherwise(
    f.when(f.col('CD_FAIXA_ETARIA_SOCIO')==f.lit(3), f.lit('21 a 30 anos')).otherwise(
      f.when(f.col('CD_FAIXA_ETARIA_SOCIO')==f.lit(4), f.lit('31 a 40 anos')).otherwise(
        f.when(f.col('CD_FAIXA_ETARIA_SOCIO')==f.lit(5), f.lit('41 a 50 anos')).otherwise(
          f.when(f.col('CD_FAIXA_ETARIA_SOCIO')==f.lit(6), f.lit('51 a 60 anos')).otherwise(
            f.when(f.col('CD_FAIXA_ETARIA_SOCIO')==f.lit(7), f.lit('61 a 70 anos')).otherwise(
              f.when(f.col('CD_FAIXA_ETARIA_SOCIO')==f.lit(8), f.lit('71 a 80 anos')).otherwise(
                f.when(f.col('CD_FAIXA_ETARIA_SOCIO')==f.lit(9), f.lit('Maiores de 80 anos')).otherwise(
                  f.lit('Não informado'))))))))))\
  .groupby('SG_UF', 'cd_sit_cadastral','ds_sit_cadastral' ,'CNAE', 'CD_FAIXA_ETARIA_SOCIO','ds_faixa_etaria_socio','ANO' ) \
  .agg(f.sum('prop_homem').alias('Masculino'), f.sum('prop_mulher').alias('Feminino')) \
  .unpivot(['SG_UF', 'cd_sit_cadastral','ds_sit_cadastral' ,'CNAE', 'CD_FAIXA_ETARIA_SOCIO','ds_faixa_etaria_socio','ANO'], ["Masculino", "Feminino"], "SEXO", "VALOR"))

In [0]:
socios=socios.toDF(*[c.lower() for c in socios.columns]).withColumnRenamed('cnae','cd_cnae').withColumnRenamed('valor','vl_socias').withColumnRenamed('sexo','ds_sexo').join(df_uf,'sg_uf','left').drop('sg_uf','ds_uf','dh_insertion_trs').withColumn('cd_sexo', f.when(f.col('ds_sexo')==f.lit('Feminino'), f.lit(2)).otherwise(f.when(f.col('ds_sexo')==f.lit('Masculino'), f.lit(1)))).select('ano','cd_uf','cd_cnae','cd_sit_cadastral','ds_sit_cadastral','cd_faixa_etaria_socio','ds_faixa_etaria_socio','cd_sexo','ds_sexo','vl_socias')

In [0]:
df = tcf.add_control_fields(socios, adf, layer="biz")

var_sink = "{adl_path}{biz}/{path_destination}/".format(adl_path=var_adls_uri, biz=dls['folders']['business'], path_destination=table["path_destination"])

In [0]:
df.write.format('parquet').save(var_sink, header = True, mode='overwrite')